In [ ]:
!pip install deepface
!pip install imutils
!pip install scikit-learn

In [4]:
from deepface import DeepFace
from deepface.detectors import FaceDetector
from PIL import Image
import cv2
import imutils
import os
import numpy
import shutil
import csv
from sklearn.metrics import f1_score

In [6]:
## GLOBALS ##
FOLDER_PICTURES = "pictures"
FOLDER_FACES = "faces"
FOLDER_CLASSIFIED_FACES = "classified_faces"
FOLDER_CLUSTERS = "clusters"
FOLDER_CLUSTERS_PICTURES = "clusters_pictures"
FOLDER_SPECIFIC_CLUSTER = "cluster_"

TEMPORARY = "temporary"

OPEN_CV = 'opencv'
MTCNN = 'mtcnn'

COSINE_SCORE_KEY = "VGG-Face_cosine"
FACE_PATH_KEY = "identity"

PICTURE_jpg_EXTENSION = ".jpg"
PICTURE_JPG_EXTENSION = ".JPG"
PICTURE_PNG_EXTENSION = ".png"
PICTURE_JPEG_EXTENSION = ".jpeg"

JUPYTER_CHECKPOINTS = ".ipynb_checkpoints"


RGB = "RGB"
AGE = "age"
GENDER = "gender"
EMOTION = "emotion"
DOMINANT_EMOTION = "dominant_emotion"

SLASH = '/'

In [3]:
## GLOBALS ##

pictures = []

pictures_and_corresponding_faces = {}

face_counter = 0

mtcnn_model = FaceDetector.build_model(MTCNN)

In [4]:
## BLURINESS DETECTION ##

def detect_blur_fft(image, size=60):
    (h, w) = image.shape
    (cX, cY) = (int(w / 2.0), int(h / 2.0))
    fft = numpy.fft.fft2(image)
    fftShift = numpy.fft.fftshift(fft)
    fftShift[cY - size:cY + size, cX - size:cX + size] = 0
    fftShift = numpy.fft.ifftshift(fftShift)
    recon = numpy.fft.ifft2(fftShift)
    magnitude = 20 * numpy.log(numpy.abs(recon))
    mean = numpy.mean(magnitude)
    return mean

In [5]:
## LOAD PICTURES INTO A LIST ##
def load_pictures():
    global pictures
    global pictures_and_corresponding_faces
    
    for picture in os.listdir(FOLDER_PICTURES):
        if (picture.endswith(PICTURE_JPG_EXTENSION) or picture.endswith(PICTURE_PNG_EXTENSION) or picture.endswith(PICTURE_JPEG_EXTENSION) or picture.endswith(PICTURE_jpg_EXTENSION)):
            full_path = FOLDER_PICTURES + SLASH + picture
            pictures.append(full_path)
            pictures_and_corresponding_faces[full_path] = []

In [6]:
## FIND FACES WHICH ARE NOT VERY BLURRY ##
def find_faces():
    global pictures
    global face_counter
    
    for picture in pictures:
        
        opencv_picture = cv2.imread(picture)
        # faces[i][0] is matrix representation of the picture, faces[i][1] is tuple (x1, y1, width, height) ==> x2=x1+width, y2=y1+height
        faces = FaceDetector.detect_faces(mtcnn_model, MTCNN, opencv_picture)
        
        max_width = 0
        max_height = 0
        
        for face in faces:
            if face[1][2] > max_width:
                max_width = face[1][2]
            if face[1][3] > max_height:
                max_height = face[1][3]
                
        for face in faces:
            
            PIL_picture = Image.open(picture)
            
            x1 = face[1][0]
            y1 = face[1][1]
            width = face[1][2]
            height = face[1][3]
            
            if 3 * width < max_width or 3 * height < max_height:
                continue
                
            PIL_face = PIL_picture.crop((x1, y1, x1 + width, y1 + height))
            PIL_face = PIL_face.convert(RGB)
            open_cv_image = numpy.array(PIL_face) 
            open_cv_image = open_cv_image[:, :, ::-1].copy()

            open_cv_image = imutils.resize(open_cv_image, width=500)
            open_cv_image_gray = cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2GRAY)
            # apply our blur detector using the FFT
            fft_mean = detect_blur_fft(open_cv_image_gray, size=60)
            
            #bilo je 12
            if fft_mean <= 9.5:
                continue
            
            face_path = FOLDER_FACES + SLASH + str(face_counter) + PICTURE_jpg_EXTENSION            
            PIL_face.save(face_path)
            pictures_and_corresponding_faces[picture].append(face_path)
            
            face_counter += 1

In [1]:
## DICTIONARY RESULT FROM THE LAST TWO FUNCTIONS ##
#pictures_and_corresponding_faces = {"pictures/BED_0105.jpg": ["faces/0.jpg", "faces/1.jpg"], "pictures/BED_0313.jpg": ["faces/2.jpg", "faces/3.jpg", "faces/4.jpg", "faces/5.jpg"], "pictures/BED_0351.jpg": ["faces/6.jpg", "faces/7.jpg", "faces/8.jpg", "faces/9.jpg"], "pictures/BED_0368.jpg": ["faces/10.jpg", "faces/11.jpg", "faces/12.jpg"], "pictures/BED_0474.jpg": ["faces/13.jpg", "faces/14.jpg", "faces/15.jpg", "faces/16.jpg"], "pictures/BED_0543.jpg": ["faces/17.jpg", "faces/18.jpg", "faces/19.jpg", "faces/20.jpg", "faces/21.jpg", "faces/22.jpg", "faces/23.jpg", "faces/24.jpg", "faces/25.jpg"], "pictures/BED_0548.jpg": ["faces/26.jpg", "faces/27.jpg", "faces/28.jpg", "faces/29.jpg", "faces/30.jpg", "faces/31.jpg"], "pictures/BED_0553.jpg": ["faces/32.jpg", "faces/33.jpg", "faces/34.jpg", "faces/35.jpg", "faces/36.jpg", "faces/37.jpg", "faces/38.jpg", "faces/39.jpg", "faces/40.jpg"], "pictures/BED_0563.jpg": ["faces/41.jpg", "faces/42.jpg", "faces/43.jpg"], "pictures/BED_0567.jpg": ["faces/44.jpg", "faces/45.jpg", "faces/46.jpg"], "pictures/BED_0572.jpg": ["faces/47.jpg", "faces/48.jpg", "faces/49.jpg", "faces/50.jpg", "faces/51.jpg", "faces/52.jpg"], "pictures/BED_0573.jpg": [], "pictures/BED_0586.jpg": ["faces/53.jpg", "faces/54.jpg", "faces/55.jpg"], "pictures/BED_0590.jpg": ["faces/56.jpg", "faces/57.jpg", "faces/58.jpg"], "pictures/BED_7822.jpg": ["faces/59.jpg"], "pictures/BED_7831.jpg": ["faces/60.jpg"], "pictures/BED_7834.jpg": ["faces/61.jpg", "faces/62.jpg", "faces/63.jpg"], "pictures/BED_7835.jpg": [], "pictures/BED_7848.jpg": [], "pictures/BED_7851.jpg": [], "pictures/BED_7854.jpg": ["faces/64.jpg", "faces/65.jpg"], "pictures/BED_7857.jpg": ["faces/66.jpg", "faces/67.jpg"], "pictures/BED_7865.jpg": ["faces/68.jpg", "faces/69.jpg", "faces/70.jpg", "faces/71.jpg", "faces/72.jpg"], "pictures/BED_7867.jpg": ["faces/73.jpg", "faces/74.jpg", "faces/75.jpg"], "pictures/BED_7878.jpg": ["faces/76.jpg", "faces/77.jpg", "faces/78.jpg", "faces/79.jpg", "faces/80.jpg"], "pictures/BED_7882.jpg": ["faces/81.jpg", "faces/82.jpg", "faces/83.jpg", "faces/84.jpg"], "pictures/BED_7890.jpg": ["faces/85.jpg", "faces/86.jpg", "faces/87.jpg", "faces/88.jpg"], "pictures/BED_7894.jpg": ["faces/89.jpg", "faces/90.jpg", "faces/91.jpg", "faces/92.jpg", "faces/93.jpg", "faces/94.jpg", "faces/95.jpg", "faces/96.jpg", "faces/97.jpg"], "pictures/BED_7900.jpg": ["faces/98.jpg", "faces/99.jpg", "faces/100.jpg", "faces/101.jpg"], "pictures/BED_7905.jpg": ["faces/102.jpg", "faces/103.jpg", "faces/104.jpg", "faces/105.jpg"], "pictures/BED_7930.jpg": ["faces/106.jpg", "faces/107.jpg", "faces/108.jpg", "faces/109.jpg"], "pictures/BED_7934.jpg": ["faces/110.jpg", "faces/111.jpg", "faces/112.jpg", "faces/113.jpg", "faces/114.jpg"], "pictures/BED_7948.jpg": ["faces/115.jpg", "faces/116.jpg", "faces/117.jpg"], "pictures/BED_7966.jpg": ["faces/118.jpg", "faces/119.jpg", "faces/120.jpg", "faces/121.jpg"], "pictures/BED_7974.jpg": ["faces/122.jpg", "faces/123.jpg", "faces/124.jpg", "faces/125.jpg", "faces/126.jpg"], "pictures/BED_7977.jpg": ["faces/127.jpg", "faces/128.jpg", "faces/129.jpg", "faces/130.jpg", "faces/131.jpg"], "pictures/BED_7982.jpg": ["faces/132.jpg", "faces/133.jpg", "faces/134.jpg"], "pictures/BED_7993.jpg": ["faces/135.jpg", "faces/136.jpg", "faces/137.jpg"], "pictures/BED_8018.jpg": ["faces/138.jpg", "faces/139.jpg"], "pictures/BED_8032.jpg": ["faces/140.jpg", "faces/141.jpg", "faces/142.jpg", "faces/143.jpg"], "pictures/BED_8043.jpg": ["faces/144.jpg", "faces/145.jpg", "faces/146.jpg", "faces/147.jpg", "faces/148.jpg"], "pictures/BED_8057.jpg": ["faces/149.jpg", "faces/150.jpg", "faces/151.jpg", "faces/152.jpg", "faces/153.jpg", "faces/154.jpg"], "pictures/BED_8070.jpg": ["faces/155.jpg", "faces/156.jpg", "faces/157.jpg"], "pictures/BED_8076.jpg": ["faces/158.jpg", "faces/159.jpg", "faces/160.jpg", "faces/161.jpg", "faces/162.jpg"], "pictures/BED_8084.jpg": ["faces/163.jpg", "faces/164.jpg", "faces/165.jpg", "faces/166.jpg"], "pictures/BED_8086.jpg": ["faces/167.jpg", "faces/168.jpg"], "pictures/BED_8090.jpg": ["faces/169.jpg", "faces/170.jpg", "faces/171.jpg"], "pictures/BED_8094.jpg": ["faces/172.jpg", "faces/173.jpg", "faces/174.jpg"], "pictures/BED_8099.jpg": ["faces/175.jpg", "faces/176.jpg", "faces/177.jpg", "faces/178.jpg"], "pictures/BED_8103.jpg": ["faces/179.jpg", "faces/180.jpg", "faces/181.jpg"], "pictures/BED_8112.jpg": ["faces/182.jpg", "faces/183.jpg", "faces/184.jpg"], "pictures/BED_8113.jpg": ["faces/185.jpg", "faces/186.jpg"], "pictures/BED_8115.jpg": ["faces/187.jpg", "faces/188.jpg", "faces/189.jpg"], "pictures/BED_8120.jpg": ["faces/190.jpg", "faces/191.jpg", "faces/192.jpg", "faces/193.jpg", "faces/194.jpg"], "pictures/BED_8127.jpg": ["faces/195.jpg", "faces/196.jpg", "faces/197.jpg", "faces/198.jpg", "faces/199.jpg"], "pictures/BED_8177.jpg": ["faces/200.jpg", "faces/201.jpg"], "pictures/BED_8180.jpg": ["faces/202.jpg", "faces/203.jpg"], "pictures/BED_8188.jpg": [], "pictures/BED_8196.jpg": ["faces/204.jpg", "faces/205.jpg", "faces/206.jpg"], "pictures/BED_8213.jpg": ["faces/207.jpg", "faces/208.jpg", "faces/209.jpg", "faces/210.jpg", "faces/211.jpg", "faces/212.jpg", "faces/213.jpg", "faces/214.jpg", "faces/215.jpg", "faces/216.jpg"], "pictures/BED_8215.jpg": ["faces/217.jpg", "faces/218.jpg", "faces/219.jpg", "faces/220.jpg"], "pictures/BED_8217.jpg": ["faces/221.jpg", "faces/222.jpg", "faces/223.jpg"], "pictures/BED_8229.jpg": ["faces/224.jpg"], "pictures/BED_8231.jpg": ["faces/225.jpg", "faces/226.jpg", "faces/227.jpg"], "pictures/BED_8237.jpg": ["faces/228.jpg", "faces/229.jpg", "faces/230.jpg", "faces/231.jpg", "faces/232.jpg"], "pictures/BED_8239.jpg": ["faces/233.jpg", "faces/234.jpg", "faces/235.jpg", "faces/236.jpg"], "pictures/BED_8249.jpg": ["faces/237.jpg", "faces/238.jpg", "faces/239.jpg", "faces/240.jpg"], "pictures/BED_8252.jpg": ["faces/241.jpg", "faces/242.jpg", "faces/243.jpg"], "pictures/BED_8263.jpg": ["faces/244.jpg", "faces/245.jpg", "faces/246.jpg", "faces/247.jpg", "faces/248.jpg", "faces/249.jpg"], "pictures/BED_8267.jpg": ["faces/250.jpg", "faces/251.jpg", "faces/252.jpg", "faces/253.jpg"], "pictures/BED_8280.jpg": ["faces/254.jpg", "faces/255.jpg", "faces/256.jpg", "faces/257.jpg"], "pictures/BED_8282.jpg": ["faces/258.jpg", "faces/259.jpg", "faces/260.jpg", "faces/261.jpg", "faces/262.jpg", "faces/263.jpg", "faces/264.jpg", "faces/265.jpg"], "pictures/BED_8299.jpg": ["faces/266.jpg", "faces/267.jpg", "faces/268.jpg", "faces/269.jpg", "faces/270.jpg", "faces/271.jpg", "faces/272.jpg"], "pictures/BED_8329.jpg": ["faces/273.jpg", "faces/274.jpg", "faces/275.jpg", "faces/276.jpg", "faces/277.jpg", "faces/278.jpg"], "pictures/BED_8332.jpg": ["faces/279.jpg", "faces/280.jpg", "faces/281.jpg", "faces/282.jpg"], "pictures/BED_8334.jpg": ["faces/283.jpg", "faces/284.jpg", "faces/285.jpg"], "pictures/BED_8338.jpg": ["faces/286.jpg", "faces/287.jpg"], "pictures/BED_8342.jpg": ["faces/288.jpg", "faces/289.jpg", "faces/290.jpg"], "pictures/BED_8347.jpg": ["faces/291.jpg", "faces/292.jpg", "faces/293.jpg"], "pictures/BED_8369.jpg": ["faces/294.jpg", "faces/295.jpg"], "pictures/BED_8370.jpg": ["faces/296.jpg", "faces/297.jpg", "faces/298.jpg"], "pictures/BED_8396.jpg": ["faces/299.jpg"], "pictures/BED_8425.jpg": ["faces/300.jpg", "faces/301.jpg", "faces/302.jpg", "faces/303.jpg", "faces/304.jpg"], "pictures/BED_8431.jpg": ["faces/305.jpg", "faces/306.jpg", "faces/307.jpg"], "pictures/BED_8435.jpg": ["faces/308.jpg", "faces/309.jpg", "faces/310.jpg", "faces/311.jpg", "faces/312.jpg", "faces/313.jpg"], "pictures/BED_8438.jpg": ["faces/314.jpg", "faces/315.jpg", "faces/316.jpg", "faces/317.jpg", "faces/318.jpg"], "pictures/BED_8445.jpg": ["faces/319.jpg", "faces/320.jpg", "faces/321.jpg", "faces/322.jpg", "faces/323.jpg", "faces/324.jpg", "faces/325.jpg", "faces/326.jpg"], "pictures/BED_8447.jpg": ["faces/327.jpg", "faces/328.jpg", "faces/329.jpg", "faces/330.jpg", "faces/331.jpg", "faces/332.jpg", "faces/333.jpg", "faces/334.jpg"], "pictures/BED_8449.jpg": ["faces/335.jpg", "faces/336.jpg", "faces/337.jpg", "faces/338.jpg", "faces/339.jpg", "faces/340.jpg", "faces/341.jpg", "faces/342.jpg", "faces/343.jpg", "faces/344.jpg", "faces/345.jpg", "faces/346.jpg", "faces/347.jpg"], "pictures/BED_8459.jpg": ["faces/348.jpg", "faces/349.jpg", "faces/350.jpg", "faces/351.jpg", "faces/352.jpg", "faces/353.jpg"], "pictures/BED_8462.jpg": ["faces/354.jpg", "faces/355.jpg", "faces/356.jpg", "faces/357.jpg", "faces/358.jpg", "faces/359.jpg", "faces/360.jpg"], "pictures/BED_8481.jpg": [], "pictures/BED_8482.jpg": [], "pictures/BED_8483.jpg": [], "pictures/BED_8492.jpg": ["faces/361.jpg", "faces/362.jpg", "faces/363.jpg", "faces/364.jpg"], "pictures/BED_8507.jpg": ["faces/365.jpg", "faces/366.jpg", "faces/367.jpg", "faces/368.jpg", "faces/369.jpg", "faces/370.jpg", "faces/371.jpg"], "pictures/BED_8509.jpg": ["faces/372.jpg", "faces/373.jpg", "faces/374.jpg", "faces/375.jpg", "faces/376.jpg"], "pictures/BED_8514.jpg": ["faces/377.jpg", "faces/378.jpg", "faces/379.jpg", "faces/380.jpg"], "pictures/BED_8532.jpg": ["faces/381.jpg", "faces/382.jpg", "faces/383.jpg", "faces/384.jpg"], "pictures/BED_8546.jpg": ["faces/385.jpg", "faces/386.jpg", "faces/387.jpg", "faces/388.jpg", "faces/389.jpg"], "pictures/BED_8548.jpg": ["faces/390.jpg", "faces/391.jpg", "faces/392.jpg", "faces/393.jpg", "faces/394.jpg"], "pictures/BED_8558.jpg": ["faces/395.jpg", "faces/396.jpg", "faces/397.jpg", "faces/398.jpg", "faces/399.jpg"], "pictures/BED_8570.jpg": ["faces/400.jpg", "faces/401.jpg", "faces/402.jpg", "faces/403.jpg", "faces/404.jpg", "faces/405.jpg", "faces/406.jpg"], "pictures/BED_8580.jpg": ["faces/407.jpg", "faces/408.jpg", "faces/409.jpg", "faces/410.jpg", "faces/411.jpg", "faces/412.jpg", "faces/413.jpg", "faces/414.jpg", "faces/415.jpg"], "pictures/BED_8586.jpg": ["faces/416.jpg"], "pictures/BED_8592.jpg": ["faces/417.jpg", "faces/418.jpg", "faces/419.jpg"], "pictures/BED_8597.jpg": ["faces/420.jpg", "faces/421.jpg", "faces/422.jpg", "faces/423.jpg", "faces/424.jpg", "faces/425.jpg"], "pictures/BED_8601.jpg": [], "pictures/BED_8605.jpg": ["faces/426.jpg", "faces/427.jpg", "faces/428.jpg"], "pictures/BED_8612.jpg": ["faces/429.jpg", "faces/430.jpg", "faces/431.jpg", "faces/432.jpg"], "pictures/BED_8619.jpg": ["faces/433.jpg", "faces/434.jpg", "faces/435.jpg"], "pictures/BED_8623.jpg": ["faces/436.jpg", "faces/437.jpg", "faces/438.jpg", "faces/439.jpg", "faces/440.jpg"], "pictures/BED_8630.jpg": ["faces/441.jpg", "faces/442.jpg", "faces/443.jpg", "faces/444.jpg", "faces/445.jpg", "faces/446.jpg", "faces/447.jpg"], "pictures/BED_8631.jpg": ["faces/448.jpg", "faces/449.jpg", "faces/450.jpg", "faces/451.jpg"], "pictures/BED_8635.jpg": ["faces/452.jpg", "faces/453.jpg", "faces/454.jpg", "faces/455.jpg"], "pictures/BED_8637.jpg": ["faces/456.jpg", "faces/457.jpg", "faces/458.jpg", "faces/459.jpg"], "pictures/BED_8639.jpg": ["faces/460.jpg", "faces/461.jpg", "faces/462.jpg"], "pictures/BED_8644.jpg": ["faces/463.jpg", "faces/464.jpg", "faces/465.jpg", "faces/466.jpg", "faces/467.jpg"], "pictures/BED_8653.jpg": ["faces/468.jpg", "faces/469.jpg", "faces/470.jpg", "faces/471.jpg", "faces/472.jpg"], "pictures/BED_8658.jpg": ["faces/473.jpg", "faces/474.jpg", "faces/475.jpg", "faces/476.jpg", "faces/477.jpg", "faces/478.jpg", "faces/479.jpg"], "pictures/BED_8674.jpg": ["faces/480.jpg", "faces/481.jpg", "faces/482.jpg", "faces/483.jpg", "faces/484.jpg"], "pictures/BED_8686.jpg": ["faces/485.jpg", "faces/486.jpg", "faces/487.jpg", "faces/488.jpg", "faces/489.jpg", "faces/490.jpg", "faces/491.jpg", "faces/492.jpg"], "pictures/BED_8724.jpg": ["faces/493.jpg", "faces/494.jpg", "faces/495.jpg"], "pictures/BED_8730.jpg": ["faces/496.jpg", "faces/497.jpg"], "pictures/BED_8731.jpg": ["faces/498.jpg", "faces/499.jpg", "faces/500.jpg"], "pictures/BED_8733.jpg": ["faces/501.jpg", "faces/502.jpg", "faces/503.jpg"], "pictures/BED_8736.jpg": ["faces/504.jpg", "faces/505.jpg", "faces/506.jpg", "faces/507.jpg"], "pictures/BED_8742.jpg": ["faces/508.jpg", "faces/509.jpg", "faces/510.jpg", "faces/511.jpg", "faces/512.jpg"], "pictures/BED_8758.jpg": ["faces/513.jpg", "faces/514.jpg"], "pictures/BED_8764.jpg": ["faces/515.jpg"], "pictures/BED_8787.jpg": ["faces/516.jpg"], "pictures/BED_8793.jpg": ["faces/517.jpg"], "pictures/BED_8802.jpg": ["faces/518.jpg", "faces/519.jpg"], "pictures/BED_8812.jpg": ["faces/520.jpg", "faces/521.jpg"], "pictures/BED_8816.jpg": [], "pictures/BED_8821.jpg": [], "pictures/BED_8851.jpg": ["faces/522.jpg", "faces/523.jpg"], "pictures/BED_8867.jpg": ["faces/524.jpg"], "pictures/BED_8902.jpg": ["faces/525.jpg"], "pictures/BED_8926.jpg": ["faces/526.jpg", "faces/527.jpg", "faces/528.jpg"], "pictures/BED_8935.jpg": ["faces/529.jpg", "faces/530.jpg"], "pictures/BED_8938.jpg": ["faces/531.jpg"], "pictures/BED_8939.jpg": ["faces/532.jpg"], "pictures/BED_8960.jpg": ["faces/533.jpg", "faces/534.jpg"], "pictures/BED_8972.jpg": ["faces/535.jpg", "faces/536.jpg"], "pictures/BED_8981.jpg": [], "pictures/BED_8986.jpg": [], "pictures/BED_8989.jpg": [], "pictures/BED_8994.jpg": [], "pictures/BED_8999.jpg": ["faces/537.jpg", "faces/538.jpg", "faces/539.jpg", "faces/540.jpg", "faces/541.jpg"], "pictures/BED_9003.jpg": ["faces/542.jpg"], "pictures/BED_9014.jpg": [], "pictures/BED_9015.jpg": [], "pictures/BED_9029.jpg": [], "pictures/BED_9034.jpg": ["faces/543.jpg"], "pictures/BED_9039.jpg": ["faces/544.jpg", "faces/545.jpg"], "pictures/BED_9045.jpg": [], "pictures/BED_9047.jpg": [], "pictures/BED_9056.jpg": [], "pictures/BED_9058.jpg": [], "pictures/BED_9062.jpg": [], "pictures/BED_9068.jpg": [], "pictures/BED_9072.jpg": ["faces/546.jpg"], "pictures/BED_9074.jpg": ["faces/547.jpg", "faces/548.jpg"], "pictures/BED_9081.jpg": [], "pictures/BED_9082.jpg": [], "pictures/BED_9087.jpg": [], "pictures/BED_9092.jpg": ["faces/549.jpg", "faces/550.jpg"], "pictures/BED_9097.jpg": ["faces/551.jpg", "faces/552.jpg"], "pictures/BED_9098.jpg": [], "pictures/BED_9102.jpg": ["faces/553.jpg"], "pictures/BED_9104.jpg": ["faces/554.jpg", "faces/555.jpg"], "pictures/BED_9107.jpg": [], "pictures/BED_9109.jpg": [], "pictures/BED_9113.jpg": ["faces/556.jpg"], "pictures/BED_9114.jpg": [], "pictures/BED_9117.jpg": [], "pictures/BED_9120.jpg": ["faces/557.jpg"], "pictures/BED_9124.jpg": ["faces/558.jpg", "faces/559.jpg", "faces/560.jpg"], "pictures/BED_9131.jpg": ["faces/561.jpg", "faces/562.jpg", "faces/563.jpg", "faces/564.jpg", "faces/565.jpg"], "pictures/BED_9145.jpg": ["faces/566.jpg", "faces/567.jpg"], "pictures/BED_9159.jpg": ["faces/568.jpg", "faces/569.jpg", "faces/570.jpg"], "pictures/BED_9172.jpg": ["faces/571.jpg", "faces/572.jpg", "faces/573.jpg"], "pictures/BED_9179.jpg": ["faces/574.jpg", "faces/575.jpg", "faces/576.jpg", "faces/577.jpg", "faces/578.jpg", "faces/579.jpg", "faces/580.jpg", "faces/581.jpg", "faces/582.jpg"], "pictures/BED_9212.jpg": ["faces/583.jpg", "faces/584.jpg", "faces/585.jpg", "faces/586.jpg", "faces/587.jpg"], "pictures/BED_9377.jpg": ["faces/588.jpg", "faces/589.jpg", "faces/590.jpg", "faces/591.jpg", "faces/592.jpg", "faces/593.jpg", "faces/594.jpg", "faces/595.jpg", "faces/596.jpg"], "pictures/BED_9409.jpg": ["faces/597.jpg", "faces/598.jpg", "faces/599.jpg", "faces/600.jpg", "faces/601.jpg", "faces/602.jpg", "faces/603.jpg", "faces/604.jpg"], "pictures/BED_9428.jpg": ["faces/605.jpg", "faces/606.jpg", "faces/607.jpg", "faces/608.jpg", "faces/609.jpg", "faces/610.jpg", "faces/611.jpg", "faces/612.jpg", "faces/613.jpg", "faces/614.jpg", "faces/615.jpg", "faces/616.jpg"], "pictures/BED_9434.jpg": ["faces/617.jpg", "faces/618.jpg", "faces/619.jpg", "faces/620.jpg", "faces/621.jpg", "faces/622.jpg", "faces/623.jpg", "faces/624.jpg"], "pictures/BED_9459.jpg": ["faces/625.jpg", "faces/626.jpg"], "pictures/BED_9464.jpg": ["faces/627.jpg", "faces/628.jpg", "faces/629.jpg", "faces/630.jpg"], "pictures/BED_9509.jpg": ["faces/631.jpg", "faces/632.jpg", "faces/633.jpg", "faces/634.jpg", "faces/635.jpg", "faces/636.jpg", "faces/637.jpg", "faces/638.jpg", "faces/639.jpg", "faces/640.jpg", "faces/641.jpg", "faces/642.jpg"], "pictures/BED_9514.jpg": ["faces/643.jpg", "faces/644.jpg", "faces/645.jpg", "faces/646.jpg"], "pictures/BED_9559.jpg": ["faces/647.jpg", "faces/648.jpg", "faces/649.jpg", "faces/650.jpg", "faces/651.jpg", "faces/652.jpg"], "pictures/BED_9716.jpg": ["faces/653.jpg", "faces/654.jpg", "faces/655.jpg", "faces/656.jpg"], "pictures/BED_9826.jpg": ["faces/657.jpg", "faces/658.jpg", "faces/659.jpg", "faces/660.jpg", "faces/661.jpg", "faces/662.jpg", "faces/663.jpg", "faces/664.jpg", "faces/665.jpg", "faces/666.jpg"], "pictures/DSC_4107.jpg": ["faces/667.jpg", "faces/668.jpg", "faces/669.jpg"], "pictures/DSC_4118.jpg": ["faces/670.jpg", "faces/671.jpg", "faces/672.jpg"], "pictures/DSC_4130.jpg": ["faces/673.jpg"], "pictures/DSC_4136.jpg": ["faces/674.jpg", "faces/675.jpg", "faces/676.jpg", "faces/677.jpg"]}

In [8]:
load_pictures()

In [10]:
find_faces()

In [61]:
## CLUSTER FACES INTO FOLDERS ##
while True:
    # pkg file is 1
    if len(os.listdir(FOLDER_FACES)[0]) > 1:
        face = os.listdir(FOLDER_FACES)[0]
        cosine_scores = DeepFace.find(img_path = FOLDER_FACES + SLASH + face,
                                  db_path = FOLDER_FACES,
                                  detector_backend = MTCNN,
                                  enforce_detection = False)

        cosine_scores = cosine_scores[cosine_scores[COSINE_SCORE_KEY] < 0.337]

        cluster_name = "cluster_" + str(face.split('.')[0])
        os.mkdir(FOLDER_CLUSTERS + SLASH + cluster_name)

        for ind in cosine_scores.index:
            try:
                os.rename(cosine_scores[FACE_PATH_KEY][ind],
                      FOLDER_CLUSTERS + SLASH + cluster_name + SLASH + cosine_scores[FACE_PATH_KEY][ind].split(SLASH)[1])
            except FileNotFoundError as e:
                continue
    else:
        break

There are  678  representations found in  representations_vgg_face.pkl
1/1 [==============================] - 0s 486ms/step
find function lasts  4.542464017868042  seconds
There are  678  representations found in  representations_vgg_face.pkl
1/1 [==============================] - 0s 335ms/step
find function lasts  3.696136951446533  seconds
There are  678  representations found in  representations_vgg_face.pkl
1/1 [==============================] - 0s 349ms/step
find function lasts  4.697712182998657  seconds
There are  678  representations found in  representations_vgg_face.pkl
1/1 [==============================] - 0s 375ms/step
find function lasts  3.75982928276062  seconds
There are  678  representations found in  representations_vgg_face.pkl
1/1 [==============================] - 0s 411ms/step
find function lasts  4.617705821990967  seconds
There are  678  representations found in  representations_vgg_face.pkl
1/1 [==============================] - 0s 280ms/step
find function las

AttributeError: 'NoneType' object has no attribute 'copy'

In [14]:
## CLUSTER ORIGINAL PICTURES INTO THEIR CORRESPONDING FOLDERS ## 

for cluster in os.listdir(FOLDER_CLUSTERS):
    if cluster != JUPYTER_CHECKPOINTS:
        directory_created = False
        current_directory_number = -1
        for face in os.listdir(FOLDER_CLUSTERS + SLASH + cluster):
            if len(os.listdir(FOLDER_CLUSTERS + SLASH + cluster)) < 2:
                   break
            for key, value in pictures_and_corresponding_faces.items():
                face_path = FOLDER_FACES + SLASH + face
                if face_path in value:
                    if not directory_created:
                        os.mkdir(FOLDER_CLUSTERS_PICTURES + SLASH + FOLDER_SPECIFIC_CLUSTER + str(face.split('.')[0]))
                        directory_created = True
                        current_directory_number = int(face.split('.')[0])
                    shutil.copy(key, 
                                FOLDER_CLUSTERS_PICTURES + SLASH + FOLDER_SPECIFIC_CLUSTER + str(current_directory_number) + SLASH + key.split('/')[1])

In [12]:
# loads and returns cnn21 model

#FOLDER_CNN21 = "cnn21"
#H5_CNN21 = "cnn21.h5"
#JSON_CNN21 = "cnn21.json"
#READ_MODE = 'r'
#from keras.models import model_from_json

#def load_cnn21():
    # load json and create model
#    json_file = open(FOLDER_CNN21 + SLASH + JSON_CNN21, READ_MODE)
#    loaded_model_json = json_file.read()
#    json_file.close()
#    cnn21 = model_from_json(loaded_model_json)
    # load weights into new model
#    cnn21.load_weights(FOLDER_CNN21 + SLASH + H5_CNN21)
#    return cnn21

In [ ]:
#    PIL_faces_array = array(PIL_faces).astype('float32')
#    predictions = cnn21.predict(PIL_faces_array)

In [38]:
#cnn21 = load_cnn21()

## PREDICT AGE, GENDER, EMOTION ##
data = []
header = ['Picture name', 'Age', 'Gender', 'Emotion']

with open('predictions.csv', 'w', encoding='UTF8') as f:    
    writer = csv.writer(f)  
    writer.writerow(header)

    for cluster in os.listdir(FOLDER_CLUSTERS):
        if cluster != ".ipynb_checkpoints":
            for picture in os.listdir(FOLDER_CLUSTERS + SLASH + cluster):
                if picture != ".ipynb_checkpoints":
                    face = FOLDER_CLUSTERS + SLASH + cluster + SLASH + picture
                    obj = DeepFace.analyze(img_path = face, actions = [AGE, GENDER, EMOTION], detector_backend=MTCNN, enforce_detection=False)
                    data_row = [picture, obj[AGE], obj[GENDER], obj[DOMINANT_EMOTION]]
                    data.append(data_row)
    writer.writerows(data)

Action: age:   0%|          | 0/3 [00:00<?, ?it/s]    

1/1 [==============================] - 1s 920ms/step


Action: gender:  33%|███▎      | 1/3 [00:02<00:04,  2.30s/it]

1/1 [==============================] - 1s 917ms/step


Action: emotion:  67%|██████▋   | 2/3 [00:03<00:01,  1.52s/it]

...

Action: gender:  33%|███▎      | 1/3 [00:07<00:14,  7.09s/it]

1/1 [==============================] - 4s 4s/step


Action: emotion:  67%|██████▋   | 2/3 [00:10<00:05,  5.14s/it]

1/1 [==============================] - 0s 150ms/step


Action: emotion: 100%|██████████| 3/3 [00:14<00:00,  4.76s/it]


In [4]:
def get_age_class(age):
    if (age < 10):
        return '0'
    elif (age < 20):
        return '1'
    elif (age < 36):
        return '2'
    elif (age < 46):
        return '3'
    return '4'

In [5]:
## GET METRICS FOR AGE, GENDER AND EMOTION PREDICTIONS ##
## REAL AGE, GENDER AND EMOTION HAD TO BE MANUALLY LABELED ##

rows = 0

gender_acc = 0
age_acc = 0
emotion_acc = 0

predicted_age_y = []
real_age_y = []
predicted_emotion_y = []
real_emotion_y = []
with open("predictions.csv") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        if row:
            rows += 1
            if (rows > 1):
                #f1
                age_class = get_age_class(int(row[1]))
                predicted_age_y.append(age_class)
                real_age_y.append(row[4])
                predicted_emotion_y.append(row[3])
                real_emotion_y.append(row[6])
                #accuracy
                if (row[2] == row[5]):
                    gender_acc += 1
                if (age_class == row[4]):
                    age_acc += 1
                if (row[3] == row[6]):
                    emotion_acc += 1
    print(gender_acc/rows)
    print(f1_score(real_age_y, predicted_age_y, average=None))
    print(f1_score(real_emotion_y, predicted_emotion_y, average=None))

0.8716981132075472
[0.         0.         0.84758364 0.41121495 0.7012987 ]
[0.42857143 0.65822785 0.82673267 0.69277108 0.64705882 0.60869565]
